In [1]:
import numpy as np
import pandas as pd
import pickle
import psutil
import random as rn
import tensorflow as tf
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [2]:
import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, History 
from keras import backend as K

Using TensorFlow backend.


In [3]:
with open('lime_analysis_data/complete_data.pkl', 'rb') as handle:
    complete_data = pickle.load(handle)

In [4]:
with open('lime_analysis_data/results.pkl', 'rb') as handle:
    lime_dict = pickle.load(handle)

In [5]:
x_train = complete_data['xtrain']
x_test = complete_data['xtest']
x_val = complete_data['xval']
y_train = complete_data['ytrain']
y_test = complete_data['ytest']
y_val = complete_data['yval']
# param_dict = complete_data['params_dict']

In [6]:
intensity_df = lime_dict['intensity_df'].copy()

# Analysis of Intensity df

In [7]:
header = intensity_df.iloc[0]
intensity_df = intensity_df[1:]
intensity_df.columns = header

In [8]:
intensity_df_trans = intensity_df.T

In [9]:
intensity_df_trans['sum_of_intensities'] = intensity_df_trans.abs().sum(axis=1)

In [10]:
intensity_df_trans.sort_values(by=['sum_of_intensities'],ascending=False,inplace=True)

In [11]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
intensity_df_trans['blah'] = min_max_scaler.fit_transform(intensity_df_trans['sum_of_intensities'].values.reshape(-1,1)) * 100

In [12]:
intensity_df_trans

,intensity0,intensity1,intensity2,intensity3,intensity4,intensity5,intensity6,intensity7,intensity8,intensity9,...,intensity282,intensity283,intensity284,intensity285,intensity286,intensity287,intensity288,intensity289,sum_of_intensities,blah
feature_name,,,,,,,,,,,,,,,,,,,,,
BsmtFinSF1,3935.76,-32065.7,-16375.2,2977.94,4001.41,44895.5,4788.87,45767.1,43845.5,2583.5,...,47963.3,44643.6,4514.2,-32929.7,-32721.8,4137.26,-12896.5,47694.7,7.774113e+06,100.000000
2ndFlrSF,31674.9,8601.23,31393.7,-27827.8,6798.15,-27557.6,30537.8,-27725,9153.32,27566.6,...,29662,7741.26,-25013.8,30667.6,-28506.7,7417.68,6355.15,-27240.3,6.821625e+06,87.747952
BsmtUnfSF,-9331.25,1533.95,1674.97,1837.28,-16698.9,-19086.1,-21299.5,-19783.7,-8349.06,-9236.02,...,-8886.1,-17617,669.965,28369.9,-18732.2,-9867.45,-19544.9,-9599.23,4.225455e+06,54.352887
3SsnPorch,-10889.9,-7844.26,-15353.8,-16897.6,-18370.8,-12261.9,-13719.6,-15482.9,-9074.01,-11627.5,...,-14217.1,-16377.6,-25085.5,-14751.2,-13379.2,-9591.9,-15390.2,-13130,4.144053e+06,53.305795
GrLivArea,1934.02,25952.6,26340.6,-7807.82,3992.06,-19848.7,2929.31,-18901.4,25085.8,2148.52,...,25492.1,895.921,-8121.55,25881,-18931.8,-8399.19,-8787.16,-18657.9,3.766473e+06,48.448913
MiscVal,11830.4,13339.6,10758.6,15483.8,14492,13921.7,14843.1,15095.4,14477.1,14269.1,...,16531.2,10064.4,13691.7,10132.9,13520.9,13955.1,10014,12874.5,3.720425e+06,47.856585
GarageArea,20692,2428.64,-17697.1,-6609.84,-5538.7,-17478,19388.6,2143.15,20190.8,19692.9,...,20359.8,-4382.1,2286.77,19475.1,-19096.3,-19726.2,20690.1,-6243.78,3.679725e+06,47.333051
GarageYrBlt_1937.0,4596.72,4607.99,-10189.4,-9916.44,2745.83,-1981.35,4071.19,-3184.25,9226,5962.01,...,10774.5,6240.23,15942.5,-6232.6,-14074.5,13456.4,-18614.5,7449.05,3.027338e+06,38.941266
ScreenPorch,-8258.25,12847,11761,-8226.98,-6128.74,-10308.4,-11949.9,-9911.33,-10866.7,-8409.74,...,-10454.1,-9646.62,-6573.34,-10219.9,-10117.1,-13290.1,-8412.39,-9705.05,2.922547e+06,37.593322


In [28]:
n= 56

In [30]:
intensity_df_trans[intensity_df_trans['blah'] >= n]

Index(['BsmtFinSF1', '2ndFlrSF'], dtype='object', name='feature_name')

# List of columns dict

In [12]:
list_of_cols = {}

In [13]:
list_of_cols['drop_none'] = list(x_train.columns)

# Drop 0 intensities

In [14]:
intensity_df_trans_0 = intensity_df_trans.copy()

In [15]:
intensity_df_trans_0 = intensity_df_trans_0.loc[(intensity_df_trans_0!=0).any(axis=1)]

In [16]:
list_of_cols['drop_0_columns'] = list(intensity_df_trans_0.index)

# Drop last n%

In [17]:
intensity_df_trans_n = intensity_df_trans.copy()

In [18]:
for n in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,20,25,30]:
    list_of_cols['drop_{}_pct_columns'.format(n)] = list(intensity_df_trans_n.iloc[:int(np.ceil(intensity_df_trans_n.shape[0] * (1-(n/100)))),:].index)

In [26]:
list_of_cols.keys()

dict_keys(['drop_none', 'drop_0_columns', 'drop_1_pct_columns', 'drop_2_pct_columns', 'drop_3_pct_columns', 'drop_4_pct_columns', 'drop_5_pct_columns', 'drop_6_pct_columns', 'drop_7_pct_columns', 'drop_8_pct_columns', 'drop_9_pct_columns', 'drop_10_pct_columns', 'drop_11_pct_columns', 'drop_12_pct_columns', 'drop_13_pct_columns', 'drop_14_pct_columns', 'drop_15_pct_columns', 'drop_20_pct_columns', 'drop_25_pct_columns', 'drop_30_pct_columns'])

# Training with selected columns

In [19]:
# Model Training
def get_search_space():    
    space = {'num_layers': hp.choice('num_layers',['one_hidden', 'two_hidden']),
                'units1': hp.choice('units1', [32, 64, 128, 256,512]),
                'units2': hp.choice('units2', [32, 64, 128, 256,512]),
                'dropout1': hp.uniform('dropout1', .25,.75),
                'dropout2': hp.uniform('dropout2',  .25,.75),
                'batch_size' : hp.choice('batch_size', [16,32,64,128]),
                'nb_epochs' :  500,
                'optimizer': hp.choice('optimizer',['rmsprop', 'adam', 'nadam','sgd']),
                'activation': hp.choice('activation',['relu','sigmoid']),
                'early_stop_rounds': hp.choice('early_stop_rounds',[10,20,30,40,50]),
            }
    return space

In [20]:
def create_model(params):    
#     x_train_temp = x_train.copy()
#     x_test_temp = x_test.copy()
#     y_train_temp = y_train.copy()
#     y_test_temp = y_test.copy()
    model = Sequential()
    model.add(Dense(params['units1'], input_shape=(x_train_temp.shape[1],)))
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dropout1']))
    if(params['num_layers'] == 'two_hidden'):
        model.add(Dense(params['units2']))
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dropout2']))
    model.add(Dense(1))
    model.add(Activation('linear'))
    model.compile(loss='mse', metrics=['mse'],
                  optimizer=params['optimizer'])
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=params['early_stop_rounds'])
    history = History()
    model.fit(x_train_temp, y_train_temp,
              batch_size=params['batch_size'],
              epochs=500,
              callbacks=[early_stop, history],
              verbose=0,
              validation_data=(x_valid_temp,y_valid_temp)) 
    [loss, mse] = model.evaluate(x_valid_temp,y_valid_temp, verbose=0)
    global num
    mem = psutil.virtual_memory()
    if(np.isnan(mse)):
        print("{}) Validation set root mean sq. error: NaN".format(num),"\tAvailable Mem:",(mem.available/1024)/1024,"mb")
        num = num + 1
        return {'loss': np.inf, 'status': STATUS_OK, 'model': model,'params':params}
    print("{}) Validation set root mean sq. error: {:7.2f}".format(num,mse**0.5),"\tAvailable Mem:",(mem.available/1024)/1024,"mb")
    num = num + 1
    return {'loss': loss**0.5, 'status': STATUS_OK, 'model': model, 'params':params}

In [21]:
def get_best_model(label_col):
#     global x_train, x_test, x_valid, y_train, y_test, y_valid
#     input_df, x_train, x_test, x_valid, y_train, y_test, y_valid = data(csv_name=csv_name,label_col=label_col,num_features=num_features)
    trials=Trials()
    space = get_search_space()
    print("Selecting the best network architecture specifically for your data...")
    best = fmin(create_model, space, algo=tpe.suggest, max_evals=50, trials=trials)
    best_model = trials.best_trial['result']['model']
#     scaled_feature_df = pd.concat([x_train,x_test])
#     label_df = pd.concat([y_train,y_test])
#     pred_df = make_predictions(model=best_model,df=scaled_feature_df)
#     output_df = pd.merge(input_df,pred_df['predictions'].to_frame(),left_index=True,right_index=True)
    return best_model, trials

In [22]:
label_col = 'SalePrice'

In [ ]:
import os
os.environ['PYTHONHASHSEED'] = '0'

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(42)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(12345)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of
# non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed

tf.set_random_seed(1234)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

resultant_dict = {}
num =1 
for n,(key,cols) in enumerate(list_of_cols.items()):
    print(n,key,len(cols))
    global x_train_temp,y_train_temp,x_valid_temp,y_valid_temp, x_test_temp, y_test_temp
    x_train_temp = x_train[cols].copy()
    y_train_temp = y_train.copy()
    x_valid_temp = x_val[cols].copy()
    y_valid_temp = y_val.copy()
    x_test_temp = x_test[cols].copy()
    y_test_temp = y_test.copy()
#     result_model = create_model(x_train_temp,y_train_temp,x_valid_temp,y_valid_temp,param_dict)
    best_model, output_trials = get_best_model(label_col)
    val_rmse = best_model.evaluate(x_valid_temp,y_valid_temp,verbose=0)[0] ** 0.5
    test_rmse = best_model.evaluate(x_test_temp,y_test_temp,verbose=0)[0] ** 0.5
    resultant_dict['{}'.format(key)] = [val_rmse,test_rmse]
    result_lime_analysis_1_param_opt = pd.DataFrame(resultant_dict).T
    result_lime_analysis_1_param_opt.columns = ['valid_rmse','test_rmse']
    result_lime_analysis_1_param_opt.to_csv('lime_analysis_data/result_lime_analysis_1_param_opt-corrected.csv')

0 drop_none 396
Selecting the best network architecture specifically for your data...
1) Validation set root mean sq. error: 48245.86 	Available Mem: 23645.96484375 mb
2) Validation set root mean sq. error: NaN 	Available Mem: 23575.30859375 mb
3) Validation set root mean sq. error: NaN 	Available Mem: 23569.6171875 mb
4) Validation set root mean sq. error: 42071.93 	Available Mem: 23559.32421875 mb
5) Validation set root mean sq. error: 203126.21 	Available Mem: 23548.10546875 mb
6) Validation set root mean sq. error: NaN 	Available Mem: 23539.0234375 mb
7) Validation set root mean sq. error: 203499.27 	Available Mem: 23528.07421875 mb
8) Validation set root mean sq. error: 43874.41 	Available Mem: 23515.60546875 mb
9) Validation set root mean sq. error: 202642.93 	Available Mem: 23505.16015625 mb
10) Validation set root mean sq. error: 42533.62 	Available Mem: 23495.66015625 mb
11) Validation set root mean sq. error: NaN 	Available Mem: 23488.0390625 mb
12) Validation set root mean s

100) Validation set root mean sq. error: 201644.04 	Available Mem: 22478.515625 mb
2 drop_1_pct_columns 393
Selecting the best network architecture specifically for your data...
101) Validation set root mean sq. error: 41840.19 	Available Mem: 22466.21875 mb
102) Validation set root mean sq. error: 203442.64 	Available Mem: 22455.578125 mb
103) Validation set root mean sq. error: 39186.96 	Available Mem: 22440.66015625 mb
104) Validation set root mean sq. error: 42879.97 	Available Mem: 22428.36328125 mb
105) Validation set root mean sq. error: NaN 	Available Mem: 22416.48828125 mb
106) Validation set root mean sq. error: 42697.14 	Available Mem: 22405.26953125 mb
107) Validation set root mean sq. error: 202076.52 	Available Mem: 22393.53515625 mb
108) Validation set root mean sq. error: 202661.41 	Available Mem: 22380.39453125 mb
109) Validation set root mean sq. error: 199385.51 	Available Mem: 22369.05078125 mb
110) Validation set root mean sq. error: 40560.36 	Available Mem: 22323.

197) Validation set root mean sq. error: NaN 	Available Mem: 21541.59375 mb
198) Validation set root mean sq. error: 201105.62 	Available Mem: 21521.0546875 mb
199) Validation set root mean sq. error: 41196.08 	Available Mem: 21517.78515625 mb
200) Validation set root mean sq. error: NaN 	Available Mem: 21513.546875 mb
4 drop_3_pct_columns 385
Selecting the best network architecture specifically for your data...
201) Validation set root mean sq. error: 202012.30 	Available Mem: 21508.1015625 mb
202) Validation set root mean sq. error: 81724.83 	Available Mem: 21492.22265625 mb
203) Validation set root mean sq. error: 201603.55 	Available Mem: 21487.75390625 mb
204) Validation set root mean sq. error: 80297.70 	Available Mem: 21483.63671875 mb
205) Validation set root mean sq. error: 203415.20 	Available Mem: 21466.125 mb
206) Validation set root mean sq. error: 57223.56 	Available Mem: 21460.4296875 mb
207) Validation set root mean sq. error: 80294.00 	Available Mem: 21422.234375 mb
20

294) Validation set root mean sq. error: 43105.34 	Available Mem: 19838.453125 mb
295) Validation set root mean sq. error: 43612.89 	Available Mem: 19832.27734375 mb
296) Validation set root mean sq. error: 82051.33 	Available Mem: 19793.515625 mb
297) Validation set root mean sq. error: 37358.32 	Available Mem: 19789.3984375 mb
298) Validation set root mean sq. error: 35984.33 	Available Mem: 19765.64453125 mb
299) Validation set root mean sq. error: 42765.61 	Available Mem: 19760.8203125 mb
300) Validation set root mean sq. error: 200605.86 	Available Mem: 19736.4296875 mb
6 drop_5_pct_columns 377
Selecting the best network architecture specifically for your data...
301) Validation set root mean sq. error: 203209.65 	Available Mem: 19732.4296875 mb
302) Validation set root mean sq. error: 87106.60 	Available Mem: 19726.4921875 mb
303) Validation set root mean sq. error: NaN 	Available Mem: 19704.125 mb
304) Validation set root mean sq. error: 43289.73 	Available Mem: 19700.15234375 m

391) Validation set root mean sq. error: 55436.08 	Available Mem: 18641.40234375 mb
392) Validation set root mean sq. error: 40858.46 	Available Mem: 18613.046875 mb
393) Validation set root mean sq. error: 38583.57 	Available Mem: 18609.05078125 mb
394) Validation set root mean sq. error: 80746.86 	Available Mem: 18606.765625 mb
395) Validation set root mean sq. error: 37060.21 	Available Mem: 18574.11328125 mb
396) Validation set root mean sq. error: 199966.14 	Available Mem: 18569.515625 mb
397) Validation set root mean sq. error: 47108.75 	Available Mem: 18538.1171875 mb
398) Validation set root mean sq. error: 76271.28 	Available Mem: 18531.8203125 mb
399) Validation set root mean sq. error: 197644.58 	Available Mem: 18500.43359375 mb
400) Validation set root mean sq. error: 41419.49 	Available Mem: 18492.63671875 mb
8 drop_7_pct_columns 369
Selecting the best network architecture specifically for your data...
401) Validation set root mean sq. error: 203098.59 	Available Mem: 1846

489) Validation set root mean sq. error: 44576.61 	Available Mem: 17261.6171875 mb
490) Validation set root mean sq. error: NaN 	Available Mem: 17258.2265625 mb
491) Validation set root mean sq. error: 67705.77 	Available Mem: 17253.75 mb
492) Validation set root mean sq. error: 42169.89 	Available Mem: 17217.390625 mb
493) Validation set root mean sq. error: 40303.27 	Available Mem: 17214.3828125 mb
494) Validation set root mean sq. error: 41075.71 	Available Mem: 17211.23828125 mb
495) Validation set root mean sq. error: 43650.70 	Available Mem: 17174.28515625 mb
496) Validation set root mean sq. error: 202558.48 	Available Mem: 17171.1796875 mb
497) Validation set root mean sq. error: NaN 	Available Mem: 17168.27734375 mb
498) Validation set root mean sq. error: 198760.31 	Available Mem: 17130.55859375 mb
499) Validation set root mean sq. error: 38997.47 	Available Mem: 17127.19921875 mb
500) Validation set root mean sq. error: 201755.74 	Available Mem: 17122.25 mb
10 drop_9_pct_col

In [34]:
resultant_dict.keys()

dict_keys(['drop_none', 'drop_0_columns', 'drop_1_pct_columns', 'drop_2_pct_columns', 'drop_3_pct_columns', 'drop_4_pct_columns', 'drop_5_pct_columns', 'drop_6_pct_columns', 'drop_7_pct_columns', 'drop_8_pct_columns', 'drop_9_pct_columns', 'drop_10_pct_columns', 'drop_11_pct_columns', 'drop_12_pct_columns', 'drop_13_pct_columns', 'drop_14_pct_columns', 'drop_15_pct_columns'])